In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

c:\Users\giuli\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
with open('matrix_list.txt', 'r') as file:
    content = file.read()

In [3]:
content_list = content.split("\n")
grouped_list = [content_list[i:i+3] for i in range(0, len(content_list), 3)]

In [4]:
samples_prefixes = [sample[0].split("_")[1] for sample in grouped_list[::2]]

In [5]:
samples_prefixes = [f'{sample}_CSF_' for sample in samples_prefixes] + [f'{sample}_PBMC_' for sample in samples_prefixes]

In [6]:
patients = [sample[0].split("_")[0] for sample in grouped_list[::1]]

In [7]:
len(samples_prefixes)

22

In [8]:
adatas = {}
for (barcode, feature, matrix), prefix in zip(grouped_list, samples_prefixes):
    adatas[prefix] = sc.read_10x_mtx(
        '../GSE194078/MTX',
        var_names='gene_symbols',
        gex_only=False,
        cache=True,
        prefix=prefix)

In [9]:
samples_prefixes

['CYG_CSF_',
 'JSB_CSF_',
 'JYJ_CSF_',
 'KHW_CSF_',
 'KJS_CSF_',
 'KSH_CSF_',
 'KYO_CSF_',
 'SYG_CSF_',
 'YYJ_CSF_',
 'YYS_CSF_',
 'YYW_CSF_',
 'CYG_PBMC_',
 'JSB_PBMC_',
 'JYJ_PBMC_',
 'KHW_PBMC_',
 'KJS_PBMC_',
 'KSH_PBMC_',
 'KYO_PBMC_',
 'SYG_PBMC_',
 'YYJ_PBMC_',
 'YYS_PBMC_',
 'YYW_PBMC_']

In [10]:
disease_dict = {
    'CYG': 'Ab-mediated IDD',
    'JSB': 'Normal',
    'JYJ': 'MS',
    'KHW': 'Ab-mediated IDD',
    'KJS': 'MS',
    'KSH': 'Ab-mediated IDD',
    'KYO': 'Normal',
    'SYG': 'Ab-mediated IDD',
    'YYJ': 'Ab-mediated IDD',
    'YYS': 'Ab-mediated IDD',
    'YYW': 'MS'
}

In [11]:
csf_adata = adatas[samples_prefixes[0]]
csf_adata.obs['patient'] = patients[0]
csf_adata.obs['patient_id'] = samples_prefixes[0].split("_")[0]
csf_adata.obs['disease'] = disease_dict[samples_prefixes[0].split("_")[0]]

for i, prefix in enumerate(samples_prefixes[1:11]):
    adata = adatas[prefix]
    adata.obs.index = adata.obs.index.str[:-1] + str(i+2)
    adata.var_names_make_unique()
    adata.obs['patient'] = patients[i+1]
    adata.obs['patient_id'] = prefix.split("_")[0]
    adata.obs['disease'] = disease_dict[prefix.split("_")[0]]
    csf_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
    
csf_adata.obs['dataset'] = 'GSE194078'
csf_adata.obs['cell_types'] = 'CSF'

C:\Users\giuli\AppData\Local\Temp\ipykernel_21416\4271032471.py:13: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  csf_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_21416\4271032471.py:13: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  csf_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_21416\4271032471.py:13: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthe

In [12]:
csf_adata

AnnData object with n_obs × n_vars = 48662 × 36601
    obs: 'patient', 'patient_id', 'disease', 'batch', 'dataset', 'cell_types'
    var: 'gene_ids', 'feature_types'

In [13]:
csf_adata.var['feature_types']

MIR1302-2HG    Gene Expression
FAM138A        Gene Expression
OR4F5          Gene Expression
AL627309.1     Gene Expression
AL627309.3     Gene Expression
                    ...       
AC141272.1     Gene Expression
AC023491.2     Gene Expression
AC007325.1     Gene Expression
AC007325.4     Gene Expression
AC007325.2     Gene Expression
Name: feature_types, Length: 36601, dtype: object

In [14]:
csf_adata.X

<48662x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 100791460 stored elements in Compressed Sparse Row format>

In [16]:
pbmc_adata = adatas[samples_prefixes[11]]
pbmc_adata.obs['patient'] = patients[11]
pbmc_adata.obs['patient_id'] = samples_prefixes[11].split("_")[0]
pbmc_adata.obs['disease'] = disease_dict[samples_prefixes[11].split("_")[0]]

for prefix in samples_prefixes[11:]:
    adata = adatas[prefix]
    adata.obs.index = adata.obs.index.str[:-1] + str(i+2)
    adata.var_names_make_unique()
    adata.obs['patient'] = patients[11+i]
    adata.obs['disease'] = prefix.split("_")[0]
    pbmc_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)

pbmc_adata.obs['dataset'] = 'GSE194078'
pbmc_adata.obs['cell_types'] = 'PBMC'

C:\Users\giuli\AppData\Local\Temp\ipykernel_21416\1782399998.py:12: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  pbmc_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_21416\1782399998.py:12: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  pbmc_adata = csf_adata.concatenate(adata, join='outer', index_unique=None)
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\giuli\anaconda3\lib\site-packages\ann

In [17]:
pbmc_adata

AnnData object with n_obs × n_vars = 61903 × 36601
    obs: 'patient', 'patient_id', 'disease', 'batch', 'dataset', 'cell_types'
    var: 'gene_ids', 'feature_types'

In [18]:
sum(csf_adata.var.index == pbmc_adata.var.index) # over 36601

36601

In [19]:
pbmc_adata.var_names_make_unique()
csf_adata.var_names_make_unique()

In [20]:
pbmc_adata.obs.index = pbmc_adata.obs.index + '-PBMC'
csf_adata.obs.index = csf_adata.obs.index + '-CSF'

In [21]:
adata = pbmc_adata.concatenate(csf_adata, join='outer', index_unique=None)
adata.obs_names_make_unique()

C:\Users\giuli\AppData\Local\Temp\ipykernel_21416\1302245356.py:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = pbmc_adata.concatenate(csf_adata, join='outer', index_unique=None)
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\giuli\anaconda3\lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")

In [22]:
adata.obs

,patient,patient_id,disease,batch,dataset,cell_types
AAACCCACAAGACCGA-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,0,GSE194078,PBMC
AAACCCACACGGCACT-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,0,GSE194078,PBMC
AAACCCATCAGAATAG-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,0,GSE194078,PBMC
AAACGAAAGGTAAAGG-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,0,GSE194078,PBMC
AAACGAACACAGCCAC-1-PBMC,GSM5827375,CYG,Ab-mediated IDD,0,GSE194078,PBMC
...,...,...,...,...,...,...
TTTGGTTGTGTTCAGT-11-CSF,GSM5827385,YYW,MS,1,GSE194078,CSF
TTTGGTTTCAAGAAAC-11-CSF,GSM5827385,YYW,MS,1,GSE194078,CSF
TTTGTTGGTATACCCA-11-CSF,GSM5827385,YYW,MS,1,GSE194078,CSF
TTTGTTGGTTATGACC-11-CSF,GSM5827385,YYW,MS,1,GSE194078,CSF


In [23]:
adata.X

<110565x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 221207022 stored elements in Compressed Sparse Row format>

In [24]:
adata.obs.index.value_counts() # make sure that the index is unique

AAACCCACAAGACCGA-1-PBMC    1
CATTCCGGTATGCGTT-4-CSF     1
CATTCCGGTAATGTGA-4-CSF     1
CATTCCGGTAATGCTC-4-CSF     1
CATTCCGCATGACTGT-4-CSF     1
                          ..
GGGCGTTAGATTGTGA-6-PBMC    1
GGGCCATTCTGGCCGA-6-PBMC    1
GGGCCATTCGAATGCT-6-PBMC    1
GGGCCATTCCATCAGA-6-PBMC    1
TTTGTTGTCTCCTGTG-11-CSF    1
Name: count, Length: 110565, dtype: int64

In [25]:
adata.obs[adata.obs.index.value_counts()>1].index.value_counts()

C:\Users\giuli\AppData\Local\Temp\ipykernel_21416\1487988141.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  adata.obs[adata.obs.index.value_counts()>1].index.value_counts()


Series([], Name: count, dtype: int64)

In [26]:
adata.write('GSE194078.h5ad')